# TensorFlow2学习笔记

本笔记参考了下面的书籍、文献、博客或者官方说明：
* TensorFlow2官方文档：https://tensorflow.google.cn/
* 简单粗暴TensorFlow 2：https://github.com/snowkylin/tensorflow-handbook
* TensorFlow 2.0 学习笔记：https://zhuanlan.zhihu.com/p/74441082

未注明出处的代码示例，`大概`就是我自己编的，`大概`的意思就是也有极小的概率是忘记注明了。。。

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers

# Stateful Container

### Trackable

In [2]:
from tensorflow.python.training.tracking.base import Trackable

In [3]:
x = Trackable()
y = Trackable()
x._track_trackable(y, 'ccc') # x引用y，并且叫该引用命名为'ccc'，或者说x依赖y

In [4]:
x._lookup_dependency('ccc') is y  # 返回名称为'ccc'的引用

True

In [5]:
y

In [6]:
x._lookup_dependency('ccc')

In [7]:
del y

In [8]:
x._lookup_dependency('ccc')

可以看到删除y之后，不影响x对其引用。因此只要根节点x没有被回收，那么x所依赖的对象就不会被回收。

### AutoTrackable
AutoTrackabke类继承Trackable类，通过`__setattr__`和`__getattr__`属性拦截访问和设置新属性（访问和建立依赖关系）。

In [9]:
from tensorflow.python.training.tracking.tracking import AutoTrackable

In [10]:
x = AutoTrackable()
y = AutoTrackable()
x.ccc = y

In [11]:
x._lookup_dependency('ccc') is y

True

In [12]:
v = tf.Variable([1,2,3])

In [13]:
x.vvv = v

In [14]:
x._unconditional_checkpoint_dependencies

[TrackableReference(name='ccc', ref=<tensorflow.python.training.tracking.tracking.AutoTrackable object at 0x13d72f2b0>),
 TrackableReference(name='vvv', ref=<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>)]

In [15]:
v

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

### 可以被保存的对象
**tf.Variable和MutableHashTable**  
tf.Variable类和MutableHashTable类是可以被保存的对象(用于tf.train.Checkpoint)，这两个类继承自Trackable类，并且覆盖了`_gather_saveables_for_checkpoint`方法，用tf.train.Checkpoint来保存。

In [16]:
from tensorflow.python.ops.lookup_ops import MutableHashTable

In [17]:
x._gather_saveables_for_checkpoint()

{}

In [18]:
x.vvv._gather_saveables_for_checkpoint()

{'VARIABLE_VALUE': <tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>}

实际上，Checkpoint使用了ObjectGraphView类，遍历整个DAG节点，并调用`_gather_saveables_for_checkpoint`方法类收集可以被保存的对象以及它们的依赖关系并存储。

### Restore-on-Creation

In [19]:
class MyModule(tf.Module):
    def assign(self, init=tf.constant([1., 2., 3.]), name=None):
        with self.name_scope:
          self.w = tf.Variable(init)
    def operate(self, value):
        self.w.assign_add(value)

m = MyModule(name='test')
m.assign()
m.operate([1., 1., 1.])
m.w

<tf.Variable 'test/Variable:0' shape=(3,) dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

In [20]:
ckpt = tf.train.Checkpoint(module=m)
ckpt.save('data/ckpt.save.test')

'data/ckpt.save.test-1'

In [21]:
module = MyModule(name='test')
try:
    module.w
except AttributeError as e:
    print("w doesn't exist.")
else:
    print("w already exists.")

w doesn't exist.


由于没用调用assign方法，可以看到w属性是不存在的。

In [22]:
ckpt = tf.train.Checkpoint(module=module)
ckpt.restore(tf.train.latest_checkpoint('data'))

In [23]:
try:
    module.w
except AttributeError as e:
    print("w doesn't exist.")
else:
    print("w already exists.")

w doesn't exist.


可以看到由于w属性没有建立，因此restore之后，w依然是不存在的。但是当调用assign方法建立w属性的时候，restore就会起作用了，可以看到结果是restore得到的结果，并不是assign的参数所指定的`tf.constant([1., 1., 1.])`。  

**Restore-on-Creation机制就是在权重没有建立时，暂时不加载checkpoint保存的权重，一旦建立，则立即加载。**

In [24]:
module.assign(tf.constant([1., 1., 1.]))
module.w  # so you see...

<tf.Variable 'test/Variable:0' shape=(3,) dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

### tf.Module
`tf.variables`：收集所有变量；  
`tf.trainable_variables`：收集所有可训练的变量；  
`tf.submodules`：收集所有子模块。

> You can enter the name scope explicitly using `with self.name_scope:` or you can annotate methods(apart from `__init__`) with `@tf.Module.with_name_scope`.

注意使用`@tf.Module.with_name_scope`或者`with self.name_scope`，必须在`__init__`中调用`super().__init__`，以此来调用`tf.Module`类的构建函数`__init__`

In [25]:
class Dense(tf.Module):
  def __init__(self, input_features, output_features, name=None):
    super(Dense, self).__init__(name=name)
    with self.name_scope:
      self.w = tf.Variable(tf.random.normal([input_features, output_features], name='w'))
      self.b = tf.Variable(tf.zeros([output_features,]), name='b')
  @tf.Module.with_name_scope
  def __call__(self, x):
    self.test = tf.Variable([2.,3.], name='ahaha')
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

d = Dense(input_features=5, output_features=3)
d(tf.ones([6, 5]))

<tf.Tensor: id=98, shape=(6, 3), dtype=float32, numpy=
array([[1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ],
       [1.8596485, 1.5468853, 0.712854 ]], dtype=float32)>

In [26]:
d.variables[0].name

'dense/b:0'

In [27]:
d.name_scope.name

'dense/'

In [28]:
d.name

'dense'

In [29]:
d.test

<tf.Variable 'dense/ahaha:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>

# tf.function

### 基本特征

* tf.function 装饰器返回的是def_function.Function对象；
* Function对象是由一个个的ConcreteFunction函数组成；ConcreteFunction对象是由包含了FunctionGraph和structured_input_signature；
* FunctionGraph是tf.Graph的子类，strucured_input_signature是函数签名；
* 如果传入的参数是一个python值，则会对每一个遇到的pyhon值创建一个ConcreteFunction，实际上python值会成为Graph的一个固定的值，如果创建ConcreteFunction时，参数是一个python的引用，则此时引用的值就被固定在Graph中；
* 这也说明，如果是参数是可变了python值，那么，在函数中就不能运行原处改变的操作，因为该值已经被固定在Graph中了；

### 运行过程

1. 运行函数的每一行代码，代码分为两类：
  * 纯python代码；
  * tensorflow代码，如`tf.add`，以及可以转换为计算节点的python代码；  
运行的结果就是：纯python代码会与运行普通的python代码相同，tensorflow代码与可以转换为计算节点的python代码会构建为计算图。
2. 运行计算图一次
3. 基于函数的名字和输入的函数参数类型生成一个哈希值，并将计算的计算图缓存到一个哈希表中

**AutoGraph与if，while循环：**  
* for：如果iterable是张量，则转换；
* while：如果while条件是张量，则转换。

### 实例

In [30]:
@tf.function
def add(x, y):
    return tf.add(x, y)

In [31]:
add(tf.random.normal((2, 3)), tf.random.normal((3,)))

<tf.Tensor: id=119, shape=(2, 3), dtype=float32, numpy=
array([[ 3.5347419 ,  1.4418387 ,  0.48181692],
       [ 4.3130684 , -0.42558745,  0.92748874]], dtype=float32)>

In [32]:
add(tf.random.normal((2, 6)), tf.random.normal((6,)))

<tf.Tensor: id=138, shape=(2, 6), dtype=float32, numpy=
array([[ 1.4450588 , -1.1302978 , -0.365623  ,  0.01166382,  3.4731584 ,
         0.07145485],
       [-0.4821213 ,  0.20358813, -2.4397168 , -1.2038107 ,  1.6287951 ,
         0.8607738 ]], dtype=float32)>

In [33]:
add._list_all_concrete_functions_for_serialization()

In [34]:
add(3,5);add(6,9)

<tf.Tensor: id=164, shape=(), dtype=int32, numpy=15>

In [35]:
add._list_all_concrete_functions_for_serialization()[2].structured_input_signature

((6, 9), {})

In [36]:
add._list_all_concrete_functions_for_serialization()[3].structured_input_signature

((3, 5), {})

In [37]:
add._list_all_concrete_functions_for_serialization()[3]()  # 参数是python值所对应的ConcreteFunction函数不需要传入参数了，因为参数值已经固定在里面了

<tf.Tensor: id=165, shape=(), dtype=int32, numpy=8>

In [38]:
add._list_all_concrete_functions_for_serialization()

In [39]:
sig = add._list_all_concrete_functions_for_serialization()[0].structured_input_signature
sig

((TensorSpec(shape=(2, 6), dtype=tf.float32, name='x'),
  TensorSpec(shape=(6,), dtype=tf.float32, name='y')),
 {})

`.get_concrete_function`获取ConcreteFunction，奇怪的是两种方式获得ConcreteFunction并不相等

In [40]:
a = add.get_concrete_function(tf.TensorSpec(shape=[2,6], dtype=tf.float32), tf.TensorSpec(shape=[6,], dtype=tf.float32))

In [41]:
a

In [42]:
add._list_all_concrete_functions_for_serialization()[0]

tf.function只允许在第一次调用函数时，创建tf.Variable；因此典型用法应当是在`__init__`方法中设置权重为`None`，然后在`build`方法中加以判断，如果权重为`None`，则初始化权重。

In [51]:
v = None

def f(x):
    global v
    if v is None:
      v = tf.Variable(x)
    return v
f = tf.function(f)

In [52]:
f._list_all_concrete_functions_for_serialization()

[]

In [53]:
f(tf.constant([2., 3., 4.]))

<tf.Tensor: id=314, shape=(3,), dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

In [54]:
f(tf.constant([2., 3.]))

<tf.Tensor: id=322, shape=(3,), dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

当我把v重新设置成None时，导致再次调用函数f时会试图创建variable，因此抛出异常。

In [55]:
try:
    v = None
    f(tf.constant([1.,2, 3.]))
except ValueError:
    print("ValueError when create variable non-first call")
else:
    print("isn't ok?")

ValueError when create variable non-first call


正确的用法应当是：

In [35]:
class MyModule(tf.Module):
    def __init__(self, name, units=10):
        super(MyModule, self).__init__(name=name)
        self.w = None
        self.b = None
        self.units = units
    @tf.Module.with_name_scope
    def build(self, input_shape):
        if self.w is None:
            self.w = tf.Variable(tf.random.normal([input_shape[-1], self.units]))
        if self.b is None:
            self.b = tf.Variable(tf.random.normal([self.units, ]))
    def call(self, input):
        return tf.matmul(input, self.w) + self.b
    @tf.function
    def __call__(self, input):
        self.build(input.shape)
        return self.call(input)

In [38]:
m = MyModule('testModule')
input = tf.random.normal([5,3])
m(input).shape

TensorShape([5, 10])

In [47]:
m.__call__._list_all_concrete_functions_for_serialization()[0].structured_input_signature

((TensorSpec(shape=(5, 3), dtype=tf.float32, name='input'),), {})

如果注释掉`build`方法中的两个`if`判断语句，导致`ValueError when create variable non-first call`

### 可变类型作为函数的参数

In [59]:
@tf.function
def f(x):
    print(x)
    # 这一行会导致错误，也就是说参数是可变类型的原处操作会导致运行错误
    # x.append(100) 
    return x[-1] + 100

In [60]:
x = [1.,2.]

In [61]:
f(x)

[1.0, 2.0]


<tf.Tensor: id=441, shape=(), dtype=float32, numpy=102.0>

In [62]:
f.get_concrete_function(x)()

<tf.Tensor: id=442, shape=(), dtype=float32, numpy=102.0>

In [63]:
f._list_all_concrete_functions_for_serialization()[0].structured_input_signature

(([1.0, 2.0],), {})

可以看到上面的例子说明：python的可变类型作为参数时，除了不能用原处操作的方法外，其他的和python值作为参数时是相同的。

下面这个例子来自于TensorFlow 2官方文档：

In [64]:
l = [] 
@tf.function 
def f(x): 
  for i in x: 
    l.append(i + 1)    # Caution! Will only happen once when tracing 
f(tf.constant([1, 2, 3])) 
l

[<tf.Tensor 'add:0' shape=() dtype=int32>]

In [65]:
f._list_all_concrete_functions_for_serialization()[0]

In [66]:
l = []
@tf.function
def f(a):
    for i in range(a):
        l.append(0)  # 只会在构建计算图时运行一次
        tf.print(a)  # 会成为计算图的一个计算节点，每次调用都会运行

In [67]:
f(3)
l

3
3
3


[0, 0, 0]

In [68]:
f(3)  # 第二次调用并不会改变list的值，因为第二次只会运行计算图
l

3
3
3


[0, 0, 0]

### 自定义类的序列化

In [70]:
class Person:
    def __init__(self, age):
        self.age = age

@tf.function
def f(year, p):
    print(year)
    return p.age + year

p = Person(100)

In [71]:
f(1, p)

1


<tf.Tensor: id=546, shape=(), dtype=int32, numpy=101>

In [72]:
f(2, p)

2


<tf.Tensor: id=551, shape=(), dtype=int32, numpy=102>

In [73]:
f(2,p)

<tf.Tensor: id=552, shape=(), dtype=int32, numpy=102>

In [81]:
f.get_concrete_function(2,p).structured_input_signature

((2, <tensorflow.python.framework.func_graph.UnknownArgument at 0x13e59f0b8>),
 {})

可能是由于Person类并没有序列化，因此导致`_list_all_concrete_functions_for_serialization`并不能获取`ConcreteFunction`

In [82]:
f._list_all_concrete_functions_for_serialization()

[]

In [91]:
@tf.function
def concat_with_padding():
    x = tf.zeros([5, 10])
    tf.print(x.shape)
    x = x[:4]
    tf.print(x.shape)
    for i in tf.range(4):
        x = tf.concat([x[:i], tf.ones([1, 10])], axis=0) # 循环时张量形状不能改变
        tf.print(x.shape)
        x.set_shape([4, 10])
        tf.print(x.shape)
    return x
concat_with_padding()

TensorShape([5, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])
TensorShape([None, 10])
TensorShape([4, 10])


<tf.Tensor: id=1095, shape=(4, 10), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

# tf.data

In [104]:
a = tf.data.Dataset.range(1, 4)  # ==> [ 1, 2, 3]
b = tf.data.Dataset.range(4, 5)  # ==> [ 4,]
c = a.concatenate(b)
list(iter(c))

[<tf.Tensor: id=1113, shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: id=1114, shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: id=1115, shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: id=1116, shape=(), dtype=int64, numpy=4>]

In [105]:
a = tf.data.Dataset.from_tensor_slices((tf.random.normal([4, 3]), [99., 0, 1, 0]))
next(iter(a.enumerate()))

(<tf.Tensor: id=1135, shape=(), dtype=int64, numpy=0>,
 (<tf.Tensor: id=1136, shape=(3,), dtype=float32, numpy=array([-0.31450528,  0.25965682,  0.49548307], dtype=float32)>,
  <tf.Tensor: id=1137, shape=(), dtype=float32, numpy=99.0>))

##### 还有这个不同，啊哈

In [76]:
a = tf.data.Dataset.from_tensor_slices([ [1, 2, 3], [4, 5, 6], [7, 8, 9] ])
next(iter(a))

<tf.Tensor: id=839, shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [77]:
a = tf.data.Dataset.from_tensor_slices(( [1, 2, 3], [4, 5, 6], [7, 8, 9] ))
next(iter(a))

(<tf.Tensor: id=849, shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: id=850, shape=(), dtype=int32, numpy=4>,
 <tf.Tensor: id=851, shape=(), dtype=int32, numpy=7>)

# API

## tf.TensorArray

In [41]:
x = tf.TensorArray(dtype=tf.float32, size=3, infer_shape=False, clear_after_read=False)
a = tf.random.normal([3,2,2])
x.unstack(a)

In [28]:
x.read(0)

<tf.Tensor: id=76, shape=(2, 2), dtype=float32, numpy=
array([[ 1.2721593 , -1.9487722 ],
       [-0.03961323, -1.1776539 ]], dtype=float32)>

In [24]:
x.stack().shape

TensorShape([3, 2, 2])

In [25]:
x.gather([1,2])

<tf.Tensor: id=70, shape=(2, 2, 2), dtype=float32, numpy=
array([[[-0.5454636 ,  0.20041323],
        [ 0.37817836, -0.14790031]],

       [[ 1.2721593 , -1.9487722 ],
        [-0.03961323, -1.1776539 ]]], dtype=float32)>

In [27]:
y = x.scatter([2,1,0], a)
y.read(0)

<tf.Tensor: id=76, shape=(2, 2), dtype=float32, numpy=
array([[ 1.2721593 , -1.9487722 ],
       [-0.03961323, -1.1776539 ]], dtype=float32)>

In [29]:
a = tf.random.normal([5,6])
x.split(a, [1,2,2]) # 长度分别是1，2，2

In [31]:
x.read(0)

<tf.Tensor: id=87, shape=(1, 6), dtype=float32, numpy=
array([[-0.01972488, -1.3749392 , -0.91153395, -0.8125202 , -1.0077246 ,
         2.24428   ]], dtype=float32)>

In [32]:
x.read(1)

<tf.Tensor: id=88, shape=(2, 6), dtype=float32, numpy=
array([[ 0.20377028, -0.40198714, -0.15850258,  0.5587745 ,  1.2975644 ,
        -0.34813213],
       [-0.7515367 , -1.2006198 , -1.1441168 , -0.38421804, -1.7521937 ,
        -0.37348786]], dtype=float32)>

## tf.save_model

使用`tf.function`一章中的`MyModule`类的实例`m`展示

In [49]:
tf.saved_model.save(m, "data/modelDir")
tf.saved_model.load("data/modelDir")

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x115467dd8>

## tf.train

#### tf.train.Checkpoint

Checkpoint只保存模型的参数，不保存模型的计算过程，因此一般用于在具有模型源码的时候恢复之前训练好的模型参数。
```python3
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.save(save_path_with_prefix)
```
* 这里tf.train.Checkpoint接受的参数比较特殊，是一个\*\*kwargs。具体而言，是一系列键值对，键名可以随便起，值为需要保存的对象。
* `save_path_with_prefix`是保存文件的目录+前缀。例如在`checkpoint.save("./save/model.ckpt")`，在save目录下会建立三个文件：`checkpoint, model.ckpt-1.index, model.ckpt-1.data-00000-of-00001`，这些文件记录了变量信息。`checkpoint.save`可以运行多次，每次运行都会得到一个`.index`文件和`.data`文件，序号一次累加。

继续训练模型可以用一下方式实现：
```
checkpoint = tf.train.Checkpoint(myAwesomeModel=model, myAwesomeOptimizer=optimizer)
checkpoint.save(save_path_with_prefix)
model_to_be_restored = MyModel() 
checkpoint = tf.train.Checkpoint(myAwesomeModel=model_to_be_restored)
checkpoint.restore(save_path_with_prefix_and_index)
```
* `save_path_with_prefix_and_index`是之前保存到文件的目录+前缀+编号。例如，调用`checkpoint.restore("./save/model.ckpt-1")`，序号为1的文件来恢复模型。

```
tf.train.latest_checkpoint(save_path)
```
* 返回最近一次的checkpoint的文件名，比如返回`./save/model.ckpt-10`

In [55]:
tf.train.latest_checkpoint('data')

'data/checkpoint-1'

## tf.initializer

如果深度学习模型的权重初始化得太小，那信号将在每层间传递时逐渐缩小而难以产生作用；如果权重初始化的太大，那信号将在每层间传递时逐渐放大并导致发散和失效。  
Xavier初始化器让初始化权重满足均值为0，方差为$\frac{2}{N_{in}+N_{out}}$均匀分布或者高斯分布；

* `tf.initializers.glorot_normal()(shape=[20,30])`：创建 $N_{in}=20,N_{out}=30$ 服从正态分布的的初始化权重；
* `tf.initializers.glorot_uniform()(shape=[20,30]`：与上面相同，只是服从的是均匀分布。

也可以通过下面的api间接实现：  
* `tf.random_normal_initializer(mean=0.0,stddev=0.05)(shape=[])`
* `tf.random_uniform_initializer(minval=-0.05, maxval=0.05)(shape=[])`

### tf.math

In [56]:
tf.math.reduce_std # 标准差
tf.math.reduce_variance # 方差
tf.math.reduce_all
tf.math.reduce_any
tf.math.reduce_logsumexp # 相当于 tf.math.log(tf.reduce_sum(tf.exp(x)))
tf.math.argmin
tf.math.argmax

<function tensorflow.python.ops.math_ops.argmax_v2(input, axis=None, output_type=tf.int64, name=None)>

### tf.GradientTape

在tf.GradientTape上下文中执行的所有操作记录下来，用于计算梯度。默认情况下，tf.GradientTape持有的资源会在调用GradientTape.gradient()方法后立即释放。要在同一计算中计算多个梯度，需要创建一个持久梯度带，这允许多次调用gradient()方法。

In [63]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
  t.watch(x)  # 由于x是常数，所以要调用调用watch方法，如果是Variable则不需要这一行
  y = x * x
  z = y * y
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = t.gradient(y, x)  # 6.0
del t  # Drop the reference to the tape
dz_dx

<tf.Tensor: id=787, shape=(), dtype=float32, numpy=108.0>

在上下文中的梯度计算也会被记录下来，因此可以实现高阶梯度计算。

In [3]:
x = tf.Variable(1.0)
with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x * x
    dy_dx = t2.gradient(y,x)
d2y_dx2 = t.gradient(dy_dx, x)

In [6]:
assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0

In [38]:
x = tf.Variable(1.)
with tf.GradientTape() as tape:
    y = x * 8.
    y = x * x
dydx = tape.gradient(y,x)

In [40]:
dydx

<tf.Tensor: id=280, shape=(), dtype=float32, numpy=2.0>

### tf.losses

```
tf.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False)
```
* 其中y_true是系数矩阵，直接的label，而不是one-hot向量
* from_logits=False时，y_pred是tf.nn.softmax输出结果，也就是每一个元素都是概率，每一行之和为1
* from_logits=True时，y_pred是上一层的输出结果，也就是说softmax(y_pred)运算在此函数内执行

两分类的交叉熵计算，可以看到下面三种计算方式结果一致：

In [6]:
tf.losses.binary_crossentropy([1,0,1], [0.9,0.3,0.7])

<tf.Tensor: id=117, shape=(), dtype=float32, numpy=0.27290332>

In [7]:
tf.reduce_mean(tf.losses.binary_crossentropy([[1],[0],[1]], [[0.9],[0.3],[0.7]]))

<tf.Tensor: id=144, shape=(), dtype=float32, numpy=0.27290332>

In [8]:
a = tf.multiply(tf.subtract(1.,[1,0,1.]), tf.math.log(tf.subtract(1.,[0.9,0.3,0.7])))
b = tf.multiply([1,0,1.], tf.math.log([0.9,0.3,0.7]))
-tf.reduce_mean(a+b)

<tf.Tensor: id=160, shape=(), dtype=float32, numpy=0.27290347>

### tf.metrics

tf.metrics.categorical_accuracy(y_true, y_pred)
* y_true是one-hot向量；y_pred是softmax输出

tf.metrics.sparse_categorical_accuracy(y_true, y_pred)
* y_true 是系数Tensor

In [53]:
a = tf.constant([1., 1, 0, 0])
b = tf.constant([0.98, 1, 0, 0.55])

In [54]:
tf.metrics.BinaryAccuracy(threshold=0.55)(a, b)

<tf.Tensor: id=652, shape=(), dtype=float32, numpy=1.0>

In [55]:
tf.metrics.binary_accuracy(a, tf.where(b>0.55, 1., 0))

<tf.Tensor: id=664, shape=(), dtype=float32, numpy=1.0>

In [56]:
tf.metrics.binary_accuracy(a, b, 0.55)

<tf.Tensor: id=671, shape=(), dtype=float32, numpy=1.0>

## tf.optimizer

```python
optimizer = tf.keras.optimizers.Adam()
grads = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradient(zip(grads, model.trainable_variables))
# .apply_gradient(grads_and_vars)
# grads_and_vars: List of (gradient, variable) pairs
```

## tf.nn

#### tf.nn.top_k

In [67]:
a = tf.random.normal([6,3])
b = tf.constant([2,1,1,0,0,1])

In [70]:
tf.nn.top_k(a,  2)

TopKV2(values=<tf.Tensor: id=748, shape=(6, 2), dtype=float32, numpy=
array([[ 0.7252414 , -0.25271565],
       [ 1.3028661 , -0.1639607 ],
       [ 1.8699104 ,  0.01289551],
       [ 0.8172665 ,  0.45867607],
       [ 1.0914992 ,  0.86236006],
       [ 0.9462557 ,  0.8255821 ]], dtype=float32)>, indices=<tf.Tensor: id=749, shape=(6, 2), dtype=int32, numpy=
array([[1, 0],
       [1, 0],
       [0, 1],
       [2, 0],
       [0, 2],
       [1, 2]], dtype=int32)>)

In [69]:
tf.nn.in_top_k(b, a,  2)

<tf.Tensor: id=746, shape=(6,), dtype=bool, numpy=array([False,  True, False, False,  True,  True])>

#### tf.nn.moment

```python
tf.nn.moments(x, axes, keep_dims=False)
# 若axes=[0,1,2]，则沿着[0,1,2]轴计算mean和variance
# keep_dims 返回的结果是否保持原来的维度
```

In [71]:
x = tf.random.normal([128, 32, 32, 64])
m, v = tf.nn.moments(x, [0,1,2], keepdims=True)
assert m.shape == [1,1,1,64]

In [80]:
# 相当于
m2 = tf.reduce_mean(x, axis=[0,1,2], keepdims=True)

In [81]:
tf.math.reduce_all(m == m2).numpy()

True

#### tf.nn.batch_normalization

```pthon
tf.nn.batch_normalization(x, mean, variance, offset, scale, variance_epsilion)
```
* mean, variance可以是`tf.nn.moments`的输出结果
* 计算公式：
```
tmp = (x-mean)/tf.sqrt(variance + variance_epsilon)
return tmp * scale + offset
```

#### tf.keras.layers.BatchNormalization

```python
tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones')
```
* axis：要标准化的特征轴
* momentum：移动平均系数
* epsilon：同上面的variance_epsilon
* scale：是否乘上scale
* center：是否加上offset
* gamma：同上面scale
* beta：同上面offset

设当前层norm的状态为：移动平均值$\mu$，标准差$\sigma$；当前mini-batch样本的均值和标准差分别为：$\mu_i,\sigma_i$；  系数为$\alpha$，也就是1 - norm.momentum；则
$$
\mu = (1-\alpha)\mu + \alpha \mu_i \\
\sigma = (1-\alpha)\sigma + \alpha\sigma_i
$$

##### [参考这篇文献](https://arxiv.org/pdf/1702.03275.pdf)，其计算过程大概如下：

In [187]:
a = tf.random.normal([6,3])

In [221]:
a

<tf.Tensor: id=1825, shape=(6, 3), dtype=float32, numpy=
array([[-0.14479367, -1.0947456 ,  0.84027725],
       [ 0.04118677, -0.22312883, -1.2298688 ],
       [-0.3011566 ,  0.17455934, -0.97229314],
       [-0.3797533 , -2.2962055 ,  0.9792077 ],
       [ 0.632281  , -0.02658297,  1.391373  ],
       [ 0.64479774, -1.5400614 ,  0.04984731]], dtype=float32)>

In [222]:
tf.nn.moments(a, axes=0) # 计算均值和方差

(<tf.Tensor: id=2307, shape=(3,), dtype=float32, numpy=array([ 0.08209366, -0.83436084,  0.1764239 ], dtype=float32)>,
 <tf.Tensor: id=2308, shape=(3,), dtype=float32, numpy=array([0.17211276, 0.7911387 , 0.97908264], dtype=float32)>)

In [254]:
norm = layers.BatchNormalization()
norm(a, training=True)  #设置权重

<tf.Tensor: id=3121, shape=(6, 3), dtype=float32, numpy=
array([[-0.54531276, -0.29256004,  0.67056483],
       [-0.09831773,  0.6867608 , -1.4205102 ],
       [-0.9211236 ,  1.1335905 , -1.1603304 ],
       [-1.110027  , -1.642482  ,  0.81089985],
       [ 1.3223488 ,  0.9075934 ,  1.2272322 ],
       [ 1.3524321 , -0.7929025 , -0.12785627]], dtype=float32)>

##### 参数

In [192]:
norm.momentum

0.99

In [193]:
norm.epsilon

0.001

In [194]:
norm.moving_mean # 初始化值为0

<tf.Variable 'batch_normalization_4/moving_mean:0' shape=(3,) dtype=float32, numpy=array([ 0.00082094, -0.00834361,  0.00176424], dtype=float32)>

In [195]:
norm.moving_variance # 初始化值为1

<tf.Variable 'batch_normalization_4/moving_variance:0' shape=(3,) dtype=float32, numpy=array([0.99172115, 0.9979114 , 0.99979085], dtype=float32)>

In [196]:
norm.beta

<tf.Variable 'batch_normalization_4/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [197]:
norm.gamma

<tf.Variable 'batch_normalization_4/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>

参数的更新计算方式与下面的计算方式相同

In [198]:
norm.momentum * 0 + tf.nn.moments(a, axes=0)[0] * (1-norm.momentum)

<tf.Tensor: id=1914, shape=(3,), dtype=float32, numpy=array([ 0.00082094, -0.00834361,  0.00176424], dtype=float32)>

In [199]:
norm.momentum * 1. + tf.nn.moments(a, axes=0)[1] * (1-norm.momentum)

<tf.Tensor: id=1926, shape=(3,), dtype=float32, numpy=array([0.99172115, 0.9979114 , 0.99979085], dtype=float32)>

##### `training=False`时计算方式

In [213]:
norm(a, False)

<tf.Tensor: id=2180, shape=(6, 3), dtype=float32, numpy=
array([[-0.14614747, -1.0869938 ,  0.83818173],
       [ 0.04051355, -0.2149022 , -1.2311465 ],
       [-0.3030826 ,  0.18300256, -0.9736725 ],
       [-0.3819669 , -2.289108  ,  0.9770573 ],
       [ 0.6337708 , -0.01824929,  1.3890597 ],
       [ 0.6463334 , -1.5325521 ,  0.04806407]], dtype=float32)>

In [214]:
#等同于下面的计算：
(a-norm.moving_mean)/(norm.moving_variance+norm.epsilon)**0.5 * norm.gamma+norm.beta

<tf.Tensor: id=2192, shape=(6, 3), dtype=float32, numpy=
array([[-0.14614747, -1.0869938 ,  0.8381817 ],
       [ 0.04051355, -0.21490222, -1.2311465 ],
       [-0.30308262,  0.18300258, -0.9736725 ],
       [-0.38196692, -2.289108  ,  0.9770572 ],
       [ 0.6337708 , -0.0182493 ,  1.3890597 ],
       [ 0.6463334 , -1.5325521 ,  0.04806407]], dtype=float32)>

##### `training=True`时的计算方式

我猜测应该是按照下面的方式计算的，不过好像数值有一点差距，暂且不管了

In [261]:
(a-tf.nn.moments(a, axes=0)[0])/(tf.nn.moments(a, axes=0)[1] + norm.epsilon)**0.5 * norm.gamma + norm.beta

<tf.Tensor: id=3217, shape=(6, 3), dtype=float32, numpy=
array([[-0.5453128 , -0.29256004,  0.6705649 ],
       [-0.09831773,  0.6867608 , -1.4205102 ],
       [-0.9211236 ,  1.1335905 , -1.1603305 ],
       [-1.110027  , -1.642482  ,  0.81089985],
       [ 1.3223488 ,  0.9075934 ,  1.2272322 ],
       [ 1.3524323 , -0.7929026 , -0.12785628]], dtype=float32)>

In [262]:
norm(a, True)

<tf.Tensor: id=3246, shape=(6, 3), dtype=float32, numpy=
array([[-0.54531276, -0.29256004,  0.67056483],
       [-0.09831773,  0.6867608 , -1.4205102 ],
       [-0.9211236 ,  1.1335905 , -1.1603304 ],
       [-1.110027  , -1.642482  ,  0.81089985],
       [ 1.3223488 ,  0.9075934 ,  1.2272322 ],
       [ 1.3524321 , -0.7929025 , -0.12785627]], dtype=float32)>

#### tf.nn.softmax

In [292]:
x = tf.Variable([[ 3., 11.,  6.],[ 6., 11., 16.]])
tf.nn.softmax(x)
# 相当于：tf.exp(x)/tf.expand_dims(tf.reduce_sum(tf.exp(x), axis=1), 1)

<tf.Tensor: id=3485, shape=(2, 3), dtype=float32, numpy=
array([[3.3310644e-04, 9.9297631e-01, 6.6906218e-03],
       [4.5094042e-05, 6.6925492e-03, 9.9326235e-01]], dtype=float32)>

#### tf.nn.softmax_cross_entropy_with_logits

```
tf.nn.softmax_cross_entropy_with_logits(labels, logits)
```
* 计算交叉熵，输入是softmax的输入，也就是说softmax的计算是在此函数的内部完成的；
* 注意返回是的一个batch的所有样本组成的向量，要求交叉熵，还要使用tf.reduce_sum；
* logits:神经网络最后一层的输出，维度是`[batch_size, num_classes]`，如果是单个样本那维度就是num_classes；
* labels:样本的实际标签，维度与上面相同

In [291]:
x = tf.Variable([[ 3., 11.,  6.],[ 6., 11., 16.]])
tf.nn.softmax(x)

<tf.Tensor: id=3476, shape=(2, 3), dtype=float32, numpy=
array([[3.3310644e-04, 9.9297631e-01, 6.6906218e-03],
       [4.5094042e-05, 6.6925492e-03, 9.9326235e-01]], dtype=float32)>

In [279]:
tf.nn.sparse_softmax_cross_entropy_with_logits([1,2], x)

<tf.Tensor: id=3361, shape=(2,), dtype=float32, numpy=array([0.0070485 , 0.00676046], dtype=float32)>

In [290]:
tf.nn.softmax_cross_entropy_with_logits(tf.one_hot([1,2], depth=3), x)

<tf.Tensor: id=3467, shape=(2,), dtype=float32, numpy=array([0.0070485 , 0.00676046], dtype=float32)>

In [289]:
- tf.reduce_sum(tf.math.log(tf.nn.softmax(x)) * tf.one_hot([1,2], depth=3), axis=1)

<tf.Tensor: id=3429, shape=(2,), dtype=float32, numpy=array([0.00704847, 0.00676045], dtype=float32)>

#### tf.nn.conv2d

```
tf.nn.conv2d(input, filters, strides, padding, data_format='NHWC', dilations=None, name=None)
```
* 第一个参数input，要求shape必须满足`[batch, in_height, in_weight, in_channels]`，具体的含义是`[训练时一个batch的图片数量，图片高度，图片宽度，图像通道数]`
* 第二个参数filters是卷积核，要求shape必须满足`[filter_height,filter_width,in_channels,out_channels]`，具体的含义是`[卷积核的高度，卷积核的宽度，图像的通道数，卷积核的个数]`
* 
* 第三个参数strides，卷积时在图像每一维的步长，这是一个一维张量，对于图片来说，`strides=[1, x, y, 1]，strides[0]==strides[3]==1`；
* 第四个参数padding，string类型的量，只能是“SAME”，“VALID”其中之一，这个值决定了不同的卷积方式， padding="SAME"表示有padding，前后补0，保证行列数不变，padding="VALID"表示不加padding；
* 第五个参数，use_cudnn_on_gpu: bool类型，是否使用cudnn加速；

In [326]:
t = layers.Conv2D(filters=2, kernel_size=[4,4])
x = tf.random.normal([6, 10, 10, 3])
t(x).shape

TensorShape([6, 7, 7, 2])

In [331]:
t.variables[0].shape # 卷积核权重的个数，每个卷积核大小为[height,width,in_channels]，卷积核的个数为out_channels

TensorShape([4, 4, 3, 2])

In [332]:
t.variables[1].shape # 卷积运算的bias个数，每个卷积核对应一个bias

TensorShape([2])

# Keras

### toy dataset

In [6]:
def process_toy(x, y):
    x = tf.cast(x, tf.float32)/255.0
    y = tf.cast(y, tf.int64)
    return x,y
def toy_dataset(n):
    (x,y), _ = keras.datasets.mnist.load_data()
    idx = np.random.choice(np.arange(x.shape[0]), n, replace=False)
    x,y = x[idx], y[idx]
    x = tf.expand_dims(x, 3)
    train_data = tf.data.Dataset.from_tensor_slices((x, y))
    train_data = train_data.map(process_toy).repeat()
    return train_data.shuffle(64).batch(32)

## 函数式API

In [7]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=8, kernel_size=[4,4], activation='relu')(inputs)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name='KerasFunctionAPIModel')

## Sequential

In [8]:
model = keras.Sequential()
model.add(layers.Conv2D(filters=8, kernel_size=[4,4], activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [104]:
train_data = toy_dataset(100)
test_data = toy_dataset(100)
model.compile(optimizer=tf.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=2, steps_per_epoch=3, validation_data=test_data, validation_steps=3)

Train for 3 steps, validate for 3 steps
Epoch 1/2
3/3 [==============================] - 1s 283ms/step - loss: 3.5683 - accuracy: 0.1562 - val_loss: 4.0477 - val_accuracy: 0.1250
Epoch 2/2
3/3 [==============================] - 0s 19ms/step - loss: 2.7161 - accuracy: 0.1458 - val_loss: 3.2233 - val_accuracy: 0.2083


## subclass

为了把Layer在此一并讲清楚，先定制一个Linear层：

In [568]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, name="MyDense"):
        super(MyDense, self).__init__(name=name)
        self.units = units
    def build(self, input_shape):
        self.w = self.add_weight("w", shape=[int(input_shape[-1]), self.units],
                                 initializer=tf.initializers.glorot_normal(),
                                 trainable=True, regularizer=keras.regularizers.l1(0.001),
                                 )
        self.b = self.add_weight("b", shape=[self.units, ],
                                 initializer=tf.initializers.glorot_uniform(),
                                 trainable=True,
                                 )
    @tf.function
    def call(self, inputs):
        y = tf.add(tf.matmul(inputs, self.w) , self.b)
        return y

In [569]:
class MyModel(tf.keras.Model):
    def __init__(self, name="MyModel", **kwargs):
        super(MyModel, self).__init__(name=name, **kwargs)
        self.conv = layers.Conv2D(filters=8, kernel_size=3, activation='relu', name='conv')
        self.flatten = layers.Flatten(name='flatten')
        self.mydense = MyDense(64, "mydense")
        self.dense = layers.Dense(32, activation='relu',
                                  use_bias=True,
                                  bias_initializer=tf.initializers.glorot_uniform(),
                                  kernel_regularizer=keras.regularizers.l2(0.01),
                                  bias_regularizer=keras.regularizers.l2(0.01),
                                  name="dense")
        self.dropout = tf.keras.layers.Dropout(0.5, name='dropout')
        self.y = layers.Dense(10, 'softmax', name='y')
    @tf.function
    def call(self, inputs, training=False):
        conv = self.conv(inputs)
        flatten = self.flatten(conv)
        mydense = self.mydense(flatten)
        dense = self.dense(mydense)
        dropout = self.dropout(dense, training=training)
        y = self.y(dropout)
        return y

In [570]:
model = MyModel("HHH")
train_data = toy_dataset(100)
a = next(iter(train_data))[0]
model(a).shape

TensorShape([32, 10])

In [571]:
tf.saved_model.save(model, "test/minimodel")

W0214 17:21:21.730880 4706063808 save_impl.py:77] Skipping full serialization of Keras model <__main__.MyModel object at 0x171a33908>, because its inputs are not defined.


In [572]:
model.summary()

Model: "HHH"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv (Conv2D)                multiple                  80        
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
mydense (MyDense)            multiple                  346176    
_________________________________________________________________
dense (Dense)                multiple                  2080      
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
y (Dense)                    multiple                  330       
Total params: 348,666
Trainable params: 348,666
Non-trainable params: 0
_________________________________________________________

In [573]:
model.losses

[<tf.Tensor: id=91185, shape=(), dtype=float32, numpy=5.4383807>,
 <tf.Tensor: id=91193, shape=(), dtype=float32, numpy=0.42587158>,
 <tf.Tensor: id=91201, shape=(), dtype=float32, numpy=0.01173897>]

In [574]:
model.get_layer('mydense').losses

[<tf.Tensor: id=91209, shape=(), dtype=float32, numpy=5.4383807>]